## Шаблоны проектирования и Python

Шаблон проектирования или паттерн (англ. design pattern) в разработке программного обеспечения — повторяемая архитектурная конструкция, представляющая собой решение проблемы проектирования в рамках некоторого часто возникающего контекста.

Обычно шаблон не является законченным образцом, который может быть прямо преобразован в код; это лишь пример решения задачи, который можно использовать в различных ситуациях. Объектно-ориентированные шаблоны показывают отношения и взаимодействия между классами или объектами, без определения того, какие конечные классы или объекты приложения будут использоваться.

«Низкоуровневые» шаблоны, учитывающие специфику конкретного языка программирования, называются идиомами. Это хорошие решения проектирования, характерные для конкретного языка или программной платформы, и потому не универсальные.

На наивысшем уровне существуют архитектурные шаблоны, они охватывают собой архитектуру всей программной системы.

Алгоритмы по своей сути также являются шаблонами, но не проектирования, а вычисления, так как решают вычислительные задачи.

[Википедия](https://ru.wikipedia.org/wiki/%D0%A8%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F)

#### Будьте осторожны

- Шаблоны проектирования — не «серебряная пуля».
- Не пытайтесь внедрять их принудительно, последствия могут быть негативными. Помните, что шаблоны — это способы решения, а не поиска проблем. Так что не перемудрите.
- Если применять их правильно и в нужных местах, они могут оказаться спасением. В противном случае у вас будет ещё больше проблем.

[Хабр](https://habr.com/ru/company/mailru/blog/325492/)


Много разных шаблонов на Питоне (и других языках) можно найти [здесь](https://refactoring.guru/ru/design-patterns/python).

Какие шаблоны проектирования мы сегодня разберем:
- [синглетон](#singleton);
- [синглетон на метаклассах](#singletonmeta);
- [фасад](#facade)
- [внедрение зависимостей (не шаблон)](#injection)
- [абстрактная фабрика](#abstractfactory);
- [состояние](#state)
- [строитель](#builder)

<a name="singleton"></a>
### Синглетон

Представим себе ситуацию, когда нам нужно, чтобы в системе находился только один объект данного типа. Реализовать подобное можно при помощи синглетона.

Допустим, мы считаем, что загружать память несколькими словарями морфологии/копиями корпуса/наборами данных/продолжить по своему усмотрению - непозволительная роскошь. Аналогично мы можем считать, что доступ к какому-то ресурсу должен осуществляться только через один-единственный интерфейсный объект без создания его копий: система логирования, чтение и запись в некоторый файл, система контроля доступа и т.д. 

В этом случае нам поможет шаблон **Синглетон (Одиночка)**.

Одна из его реулизаций выглядит следующим образом.

In [26]:
class Logger(object): # Явно прописывать наследование от object не обязательно.
    
    # Один журнал на всех.
    log = [] 
    
    # Переопределим оператор выделения объекта.
    def __new__(cls, *args, **kwargs):
        # Проверим есть ли у класса этот атрибут.
        if not hasattr(cls, '_logger'):
            # Просим родительский класс создать объект типа, переданного в cls, то есть Logger.
            cls._logger = super(Logger, cls).__new__(cls, *args, **kwargs)
        # Возвращаем созданный (только что или ранее) объект.
        return cls._logger
    
    # Эту функцию можно было и не создавать.
    def __init__(self):
        pass # Если создавать журнал для каждого объекта, то у них будут разные журналы.
        
    # Функция журналирования.
    def logIt(self, s: str):
        Logger.log.append(s)

In [27]:
a=Logger()
print(a.log)
a.logIt("first")
b=Logger()
print(b.log)
b.logIt("second")
a.logIt("third")
print(b.log)


[]
['first']
['first', 'second', 'third']


<a name="singletoneta"></a>
### Синглетон на метаклассах

Вместо такого подхода можно использовать **метаклассы**.

Метакласс - это тип, используемый для создания других объектов.

Создадим синглетон, который будет выдавать очередной свободный идентификатор.

In [92]:
# Метакласс для которого переопределяется оператор круглые скобки.
class CounterMeta(type):
    __instance = None
    
    def __call__(self):
        if CounterMeta.__instance is None:
            CounterMeta.__instance = super().__call__()
            # Вот так self.__instance = type(self)() - нельзя, потому что так мы вызываем сами себя.
            # Поэтому используем базовый класс
        return CounterMeta.__instance
    
# Сообщаем какой будет метакласс для данного класса.
class Counter(metaclass=CounterMeta):
    
    # Счетчик, хранит очередной идентификатор.
    __free_id = 0
    
    # Альтернативный вариант для хитрого финта ушами.
    def __init__(self):
        self.__private = 0
        
    # Выдает очередной свободный идентификатор и сдвигается на следующее значение.
    def get_id(self) -> int:
        new_id = Counter.__free_id
        Counter.__free_id += 1
        return new_id

In [93]:
a = Counter()
print(type(a))
print(a.get_id())
b = Counter()
print(b.get_id())
print(a.get_id())
print(Counter().get_id())

<class '__main__.Counter'>
0
1
2
3


Как мы видим, объект понимает кто его метакласс и вызывает соответствующую функцию для создания. В ней можно динамически создать нужные атрибуты и добавить необходимые функции. Как можно использовать метаклассы можно посмотреть на том же [Хабре](https://habr.com/ru/post/145835/).

Но теперь вернемся к глубинам классов. Взглянем на свойства, хранимые в таком классе.

In [94]:
Counter.__dict__

mappingproxy({'_Counter__free_id': 4,
              '__dict__': <attribute '__dict__' of 'Counter' objects>,
              '__doc__': None,
              '__init__': <function __main__.Counter.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Counter' objects>,
              'get_id': <function __main__.Counter.get_id>})

Оказывается свойство `__free_id` просто прячется за новым именем. А что, если мы ему что-нибудь присвоим.

In [85]:
Counter._Counter__free_id = -1
print(Counter().get_id())
print(Counter().get_id())

-1
0


А теперь посмотрим на `__private`. Так как он хранится в объекте, то на объект и посмотрим.

In [86]:
c = Counter()
print(c.__dict__)
c.__private = -1
print(c.__dict__)

{'_Counter__private': 0}
{'_Counter__private': 0, '__private': -1}


Он, оказывается, тоже хранится в новой переменной, в которую включено имя класса. Правильное присвоение тоже позволит менять это "спрятанное" свойство.


<div style="background:#eef; width:75%;"><i>Nothing is really private in python. No class or class instance can keep you away from all what's inside (this makes introspection possible and powerful). Python trusts you. It says "hey, if you want to go poking around in dark places, I'm gonna trust that you've got a good reason and you're not making trouble."</i></div>
[Karl Fast](https://mail.python.org/pipermail/tutor/2003-October/025932.html)


Для завершения давайте создадим еще один счетчик, но пусть он возвращает строковые идентификаторы.

А что будет, если в дочернем классе переопределить свойство @property как обычное свойство класса?

Это вызовет ошибку.

In [1]:
class A1:
    @property
    def bbb():
        return 1
    
class B2(A1):
    
    def __init__(self):
        self.bbb = 2

In [2]:
ccc = B2()
ccc.bbb

AttributeError: can't set attribute

In [311]:
# Метакласс для которого переопределяется оператор круглые скобки.
class ChrCounterMeta(type):
    __instance = None
    
    def __call__(self):
        if ChrCounterMeta.__instance is None:
            ChrCounterMeta.__instance = super().__call__()
            # Вот так self.__instance = type(self)() - нельзя, потому что так мы вызываем сами себя.
            # Поэтому используем базовый класс
        return ChrCounterMeta.__instance
    
# Сообщаем какой будет метакласс для данного класса.
class ChrCounter(metaclass=ChrCounterMeta):
    
    # Счетчик, хранит очередной идентификатор.
    __free_id = 'A'
    
    # Выдает очередной свободный идентификатор и сдвигается на следующее значение.
    def get_id(self) -> int:
        
        new_id = ChrCounter.__free_id
        pos = len(ChrCounter.__free_id) - 1
        while pos >= 0:
            if ChrCounter.__free_id[pos] == 'Z':
                if pos == 0:
                    ChrCounter.__free_id = 'A'*(len(ChrCounter.__free_id)+1)
                    break
                else:
                    ChrCounter.__free_id = ChrCounter.__free_id[:pos] + \
                                           'A' + ChrCounter.__free_id[pos+1:]
                    pos -= 1
            else:
                ChrCounter.__free_id = ChrCounter.__free_id[:pos] + \
                                       chr(ord(ChrCounter.__free_id[pos])+1) + \
                                       ChrCounter.__free_id[pos+1:]
                break
        
        return new_id

In [312]:
d = ChrCounter()
print(d.get_id())
print(d.get_id())
print(d.get_id())
print(d.get_id())
print(d.get_id())
print(d.get_id())

A
B
C
D
E
F


In [313]:
for i in range(510):
    d.get_id()
print(d.get_id())


SW


Перед тем как пойти дальше, замечу, что в многопоточном приложении мы создадим по собственному синглетону в каждом потоке и будем пользоваться ими параллельно, то есть не добъемся своей основной цели. Но чтобы решить эту проблему сперва надо понять как разрабатывать многопоточные приложения. А мы этим займемся точно не в этом модуле.

Ну и уж коли наш глаз зацепился за `type`. Он сам по себе является метаклассом и при помощи его оператора круглые скобки можно содавать новые классы. Для этого необходимо передать следующие аргументы:<br>
type(<имя класса>, <br>
<кортеж родительских классов>, _# для наследования, может быть пустым_<br>
<словарь, содержащий атрибуты и их значения>) <br>
  
Например.

In [316]:
def get_id(self) -> int:

    new_id = self.__free_id
    pos = len(self.__free_id) - 1
    while pos >= 0:
        if self.__free_id[pos] == 'Z':
            if pos == 0:
                self.__free_id = 'A'*(len(__free_id.__free_id)+1)
                break
            else:
                self.__free_id = self.__free_id[:pos] + \
                                 'A' + self.__free_id[pos+1:]
                pos -= 1
        else:
            self.__free_id = self.__free_id[:pos] + \
                             chr(ord(self.__free_id[pos])+1) + \
                             self.__free_id[pos+1:]
            break

    return new_id

def init(self):
    self.__free_id = 'A'

    
NewCounter = type('NewCounter', (), {'__init__': init, 'get_id': get_id})
e = NewCounter()
print(e.get_id())
print(e.get_id())
f = NewCounter()
print(f.get_id())
print(f.get_id())

A
B
A
B


<a name="facade"></a>
### Фасад

Фасад - это фактически переписывание интерфейса существующей системы под наши нужды. При этом может проводиться унификация интерфейсов. Хотя это не обязательно, мы можем просто обернуть несколько подсистем единым интерфейсом.

In [325]:
import re

In [336]:
# Заведем несколько простых, но дурацких классов для токенизации, лемматизации и дизамбигуации.
class SimpleTokenizer:
    def tokenize(self, text: str) -> list:
        return re.findall("[А-Яа-яёЁ]+", text)
    
class SimpleLemmatizer:
    
    def __init__(self):
        self.dictionary = {'мама':['Nfasn'], 'мамы':['Nfapn', 'Nfasg']}
        
    def addWord(self, word: str, analysis: list) -> None:
        if word in self.dictionary.keys():
            self.dictionary[word].extend(analysis)
        else:
            self.dictionary[word] = analysis
            
    def analyse(self, text: list) -> list:
        res = []
        for word in text:
            res.append(self.dictionary.get(word, ['none']))
        return res

class SimpleDisambiguator:
    
    def disambiguate(self, text: list) -> list:
        res = []
        for word in text:
            res.append(word[0])
        return res
            

In [337]:
# Создадим класс фасада, который будет проводить все действия по анализу текстов.
class TextAnalyser:
    def __init__(self):
        self.tok = SimpleTokenizer()
        self.lemm = SimpleLemmatizer()
        self.dis = SimpleDisambiguator()
        self.lemm.addWord('маму', ['Nfasa'])
        
    def analyze(self, text: str) -> list:
        return self.dis.disambiguate(self.lemm.analyse(self.tok.tokenize(text)))

In [338]:
# Получим простой интерфейс, для использования которого не важно знать что внутри.
analyzer = TextAnalyser()
print(analyzer.analyze('мама мамы маму'))

['Nfasn', 'Nfapn', 'Nfasa']


<a name="injection"></a>
### Внедрение зависимостей

Внедрение зависимостей не является шаблоном проектирования - это просто некоторый механизм, который позволяет удобно модифицировать поведение класса без использования наследования. При этом модификация может быть проведена в ходе жизни объекта.

Сделаем фасад для Pymorphy2, который будет отвечать нашему интерфейсу. Переделаем анализатор текста так, чтобы можно было передавать ему каким морфологическим анализатором мы хотим пользоваться.

На самом деле, можно было бы и не передавать морфологический анализатор, можно было бы просто вызывать соответствующий метод, а анализатор создавался бы в нем.

In [339]:
import pymorphy2

In [393]:
class DummyPymorphyFacade(SimpleLemmatizer):
    def __init__(self):
        super().__init__()
        self.morph = pymorphy2.MorphAnalyzer()
        
    def analyse(self, text: list) -> list:
        res = []
        for word in text:
            if word in self.dictionary.keys():
                res.append(self.dictionary.get(word, ['none']))
            else:
                wf = self.morph.parse(word)
                wf2 = []
                for form in wf:
                    wf2.append([f[0] for f in sorted(form.tag.grammemes)])
                res.append([''.join(w) for w in wf2])
        return res


In [394]:
a = DummyPymorphyFacade()
a.analyse(["мамой"])

[['Naafs']]

In [396]:
# Создадим класс фасада, который будет проводить все действия по анализу текстов.
class TextAnalyser2:
    def __init__(self):
        self.tok = SimpleTokenizer()
        self.lemm = SimpleLemmatizer()
        self.dis = SimpleDisambiguator()
        self.lemm.addWord('маму', ['Nfasa'])
        
    def analyze(self, text: str) -> list:
        return self.dis.disambiguate(self.lemm.analyse(self.tok.tokenize(text)))
    
    def setLemmatizer(self, lm: SimpleLemmatizer) -> None:
        self.lemm = lm
        

In [399]:
# Создадим два лематизатора.
a = DummyPymorphyFacade()
b = SimpleLemmatizer()
# Создадим новый анализатор.
c = TextAnalyser2()
# Анализируем и меняем анализаторы.
print(c.analyze('мамой'))
c.setLemmatizer(a)
print(c.analyze('мамой'))
c.setLemmatizer(b)
print(c.analyze('мамой'))


['none']
['Naafs']
['none']


Теперь представьте себе, что при создании объекта вы передаете ему объект, который будет читать для него данные. С одной стороны, первый объект не хранит все возможные форматы чтения данных. С другой стороны, он может быть динамично настроен на чтение любого известного нам формата.

<a name="abstractfactory"></a>
### Абстракная фабрика

Абстрактная фабрика является единой точкой создания объектов, наследуемых от единого базового класса. За счет наследования, объекты поддерживают единый интерфейс. С некоторой точки зрения, нам не важно какому именно классу принадлежит данный объект, если он обеспечивает необходимую функциональность по заданному интерфейсу.

В данном шаблоне проектирования создается базовый класс абстрактной фабрики, от которого наследуются фабрики конкретных классов. Абстрактная фабрика обеспечивает интерфейс создания объектов, производных от базового класса. Теперь для того, чтобы изменить производство объектов, нужно просто передать другую фабрику.

Пример (взято с [https://refactoring.guru/ru/design-patterns/python](https://refactoring.guru/ru/design-patterns/python))

In [320]:
from abc import ABC, abstractmethod


In [323]:
class AbstractFactory(ABC):
    """
    Интерфейс Абстрактной Фабрики объявляет набор методов, которые возвращают
    различные абстрактные продукты. Эти продукты называются семейством и связаны
    темой или концепцией высокого уровня. Продукты одного семейства обычно могут
    взаимодействовать между собой. Семейство продуктов может иметь несколько
    вариаций, но продукты одной вариации несовместимы с продуктами другой.
    """
    @abstractmethod
    def create_product_a(self):
        pass

    @abstractmethod
    def create_product_b(self):
        pass


class ConcreteFactory1(AbstractFactory):
    """
    Конкретная Фабрика производит семейство продуктов одной вариации. Фабрика
    гарантирует совместимость полученных продуктов. Обратите внимание, что
    сигнатуры методов Конкретной Фабрики возвращают абстрактный продукт, в то
    время как внутри метода создается экземпляр конкретного продукта.
    """

    def create_product_a(self):
        return ConcreteProductA1()

    def create_product_b(self):
        return ConcreteProductB1()


class ConcreteFactory2(AbstractFactory):
    """
    Каждая Конкретная Фабрика имеет соответствующую вариацию продукта.
    """

    def create_product_a(self):
        return ConcreteProductA2()

    def create_product_b(self):
        return ConcreteProductB2()

In [324]:
class AbstractProductA(ABC):
    """
    Каждый отдельный продукт семейства продуктов должен иметь базовый интерфейс.
    Все вариации продукта должны реализовывать этот интерфейс.
    """

    @abstractmethod
    def useful_function_a(self) -> str:
        pass


"""
Конкретные продукты создаются соответствующими Конкретными Фабриками.
"""


class ConcreteProductA1(AbstractProductA):
    def useful_function_a(self) -> str:
        return "The result of the product A1."


class ConcreteProductA2(AbstractProductA):
    def useful_function_a(self) -> str:
        return "The result of the product A2."


class AbstractProductB(ABC):
    """
    Базовый интерфейс другого продукта. Все продукты могут взаимодействовать
    друг с другом, но правильное взаимодействие возможно только между продуктами
    одной и той же конкретной вариации.
    """
    @abstractmethod
    def useful_function_b(self) -> None:
        """
        Продукт B способен работать самостоятельно...
        """
        pass

    @abstractmethod
    def another_useful_function_b(self, collaborator: AbstractProductA) -> None:
        """
        ...а также взаимодействовать с Продуктами Б той же вариации.

        Абстрактная Фабрика гарантирует, что все продукты, которые она создает,
        имеют одинаковую вариацию и, следовательно, совместимы.
        """
        pass


"""
Конкретные Продукты создаются соответствующими Конкретными Фабриками.
"""


class ConcreteProductB1(AbstractProductB):
    def useful_function_b(self) -> str:
        return "The result of the product B1."

    """
    Продукт B1 может корректно работать только с Продуктом A1. Тем не менее, он
    принимает любой экземпляр Абстрактного Продукта А в качестве аргумента.
    """

    def another_useful_function_b(self, collaborator: AbstractProductA) -> str:
        result = collaborator.useful_function_a()
        return f"The result of the B1 collaborating with the ({result})"


class ConcreteProductB2(AbstractProductB):
    def useful_function_b(self) -> str:
        return "The result of the product B2."

    def another_useful_function_b(self, collaborator: AbstractProductA):
        """
        Продукт B2 может корректно работать только с Продуктом A2. Тем не менее,
        он принимает любой экземпляр Абстрактного Продукта А в качестве
        аргумента.
        """
        result = collaborator.useful_function_a()
        return f"The result of the B2 collaborating with the ({result})"

In [322]:
def client_code(factory: AbstractFactory) -> None:
    """
    Клиентский код работает с фабриками и продуктами только через абстрактные
    типы: Абстрактная Фабрика и Абстрактный Продукт. Это позволяет передавать
    любой подкласс фабрики или продукта клиентскому коду, не нарушая его.
    """
    product_a = factory.create_product_a()
    product_b = factory.create_product_b()

    print(f"{product_b.useful_function_b()}")
    print(f"{product_b.another_useful_function_b(product_a)}", end="")

"""
Клиентский код может работать с любым конкретным классом фабрики.
"""
print("Client: Testing client code with the first factory type:")
client_code(ConcreteFactory1())

print("\n")

print("Client: Testing the same client code with the second factory type:")
client_code(ConcreteFactory2())

Client: Testing client code with the first factory type:
The result of the product B1.
The result of the B1 collaborating with the (The result of the product A1.)

Client: Testing the same client code with the second factory type:
The result of the product B2.
The result of the B2 collaborating with the (The result of the product A2.)

Рассмотрим чуть более приземленный пример.

Мы пишем стратегическую игру, в которой смена эпохи означает смену выпускаемых подразделений. Вместо того, чтобы создавать разные казармы для разных эпох, мы можем просто сменить фабрику классов у каждой казармы со старой на новую. Более того, мы можем проводить постепенное улучшение, сменяя фабрики классов по требованию пользователя. Используя инъекцию зависимостей мы можем менять поведение объекта.

Аналогично, в зависимости от того, какой именно вид анализа проводится даным объектом, мы можем менять его фабрику классов, сотавляя интерфейс и часть функционала неизменными.

<a name="state"></a>
### Состояние

Данный шаблон позволяет менять поведение объекта в зависимости от его состояния.

Для реализации данного шаблона используется комбинация конечного автомата и инъекции зависимости. Для смены состояний используется конечный автомат, описывающий последовательность таких смен. Для изменения поведения автомата используется инъекция нового объекта вместо старого.

Создадим класс машины, которая умеет включаться и выключаться. Создадим базовый класс состояния, от которого унаследуем состояние Включено и Выключено. Теперь во включенном состоянии машине будет присвоено значение Включено, в выключенном состоянии - Выключено, а при смене состояний мы будем осуществлять соответствующие действия и присваивать новое состояние.

In [414]:
# Базовый класс состояния.
class State:
    
    # Функция включения.
    def on(self, m):
        print("   already ON")
        
    # Функция выыключения.
    def off(self, m):
        print("   already OFF")

# Класс для включенного состояния.
class ON(State):
    # Переопределяем только выключение. В функцию передается машина, ей можно будет переключить состояние.
    def off(self, m):
        print("   going from ON to OFF")
        m.setCurrentState(OFF());

class OFF(State):
    
    # Переопределяем только включение. Аналогично, переключаем состояние.
    def on(self, m):
        print("   going from OFF to ON")
        m.setCurrentState(ON());

# Класс машины.        
class Machine:
    
    # Включена при создании.
    def __init__(self):
        self.current = ON()
        
    # Установка нового состояния
    def setCurrentState(self, state: State):
        self.current = state
    
    # Включаем. При включении будет заменено состояние, если машина была выключена.
    def on(self):
        self.current.on(self)
        
    # Выключаем. При выключении будет заменено состояние, если машина была выключена.
    def off(self):
        self.current.off(self)



In [415]:
m = Machine()
m.on()
m.off()
m.off()
m.on()

   already ON
   going from ON to OFF
   already OFF
   going from OFF to ON


<a name="builder"></a>
### Строитель

Данный шаблон позволяет создавать сложный объект по мере поступления ресурсов.

Если мы совместим данный шаблон с шаблоном Состояние, то можно будет накапливать ресурсы или тратить их на производство продукции в зависимости от текущего состояния.

**Плюсы**:
- Возможность контролировать процесс создания сложного продукта.
- Возможность получения разных представлений некоторых данных.

**Минусы**:
ConcreteBuilder и создаваемый им продукт жестко связаны между собой, поэтому при внесении изменений в класс продукта скорее всего придется соответствующим образом изменять и класс ConcreteBuilder.

Пример взят с [https://refactoring.guru/ru/design-patterns/builder/python/example#lang-features](https://refactoring.guru/ru/design-patterns/builder/python/example#lang-features)

In [417]:
from abc import ABC, abstractmethod, abstractproperty

In [ ]:
# Заведем абстрактного строителя.
class Builder(ABC):
    """
    Интерфейс Строителя объявляет создающие методы для различных частей объектов
    Продуктов.
    """

    @abstractproperty
    def product(self) -> None:
        pass

    @abstractmethod
    def produce_part_a(self) -> None:
        pass

    @abstractmethod
    def produce_part_b(self) -> None:
        pass

    @abstractmethod
    def produce_part_c(self) -> None:
        pass


# Заведем конкретного строителя.
class ConcreteBuilder1(Builder):
    """
    Классы Конкретного Строителя следуют интерфейсу Строителя и предоставляют
    конкретные реализации шагов построения. Ваша программа может иметь несколько
    вариантов Строителей, реализованных по-разному.
    """

    def __init__(self) -> None:
        """
        Новый экземпляр строителя должен содержать пустой объект продукта,
        который используется в дальнейшей сборке.
        """
        self.reset()

    def reset(self) -> None:
        self._product = Product1()

    @property
    def product(self):
        """
        Конкретные Строители должны предоставить свои собственные методы
        получения результатов. Это связано с тем, что различные типы строителей
        могут создавать совершенно разные продукты с разными интерфейсами.
        Поэтому такие методы не могут быть объявлены в базовом интерфейсе
        Строителя (по крайней мере, в статически типизированном языке
        программирования).

        Как правило, после возвращения конечного результата клиенту, экземпляр
        строителя должен быть готов к началу производства следующего продукта.
        Поэтому обычной практикой является вызов метода сброса в конце тела
        метода getProduct. Однако такое поведение не является обязательным, вы
        можете заставить своих строителей ждать явного запроса на сброс из кода
        клиента, прежде чем избавиться от предыдущего результата.
        """
        product = self._product
        self.reset()
        return product

    def produce_part_a(self) -> None:
        self._product.add("PartA1")

    def produce_part_b(self) -> None:
        self._product.add("PartB1")

    def produce_part_c(self) -> None:
        self._product.add("PartC1")

In [ ]:
class Product1():
    """
    Имеет смысл использовать паттерн Строитель только тогда, когда ваши продукты
    достаточно сложны и требуют обширной конфигурации.

    В отличие от других порождающих паттернов, различные конкретные строители
    могут производить несвязанные продукты. Другими словами, результаты
    различных строителей могут не всегда следовать одному и тому же интерфейсу.
    """

    def __init__(self) -> None:
        self.parts = []

    def add(self, part) -> None:
        self.parts.append(part)

    def list_parts(self) -> None:
        print(f"Product parts: {', '.join(self.parts)}", end="")


class Director:
    """
    Директор отвечает только за выполнение шагов построения в определённой
    последовательности. Это полезно при производстве продуктов в определённом
    порядке или особой конфигурации. Строго говоря, класс Директор необязателен,
    так как клиент может напрямую управлять строителями.
    """

    def __init__(self) -> None:
        self._builder = None

    @property
    def builder(self) -> Builder:
        return self._builder

    @builder.setter
    def builder(self, builder: Builder) -> None:
        """
        Директор работает с любым экземпляром строителя, который передаётся ему
        клиентским кодом. Таким образом, клиентский код может изменить конечный
        тип вновь собираемого продукта.
        """
        self._builder = builder

    """
    Директор может строить несколько вариаций продукта, используя одинаковые
    шаги построения.
    """

    def build_minimal_viable_product(self) -> None:
        self.builder.produce_part_a()

    def build_full_featured_product(self) -> None:
        self.builder.produce_part_a()
        self.builder.produce_part_b()
        self.builder.produce_part_c()

In [420]:
"""
Клиентский код создаёт объект-строитель, передаёт его директору, а затем
инициирует процесс построения. Конечный результат извлекается из объекта-
строителя.
"""

director = Director()
builder = ConcreteBuilder1()
director.builder = builder

print("Standard basic product: ")
director.build_minimal_viable_product()
builder.product.list_parts()

print("\n")

print("Standard full featured product: ")
director.build_full_featured_product()
builder.product.list_parts()

print("\n")

# Помните, что паттерн Строитель можно использовать без класса Директор.
print("Custom product: ")
builder.produce_part_a()
builder.produce_part_b()
builder.product.list_parts()

Standard basic product: 
Product parts: PartA1

Standard full featured product: 
Product parts: PartA1, PartB1, PartC1

Custom product: 
Product parts: PartA1, PartB1

Пример из стратегической игры.

In [438]:
# Куча подразделений, которые что-то там умеют.
class Unit:
    
    def __init__(self):
        self.coord = (0, 0)
        self.strenght = 10
        self.attackStr = 1
        self.name = ChrCounter().get_id()
        
    def move(self, direction):
        self.coord += direction
        
    def attack(self, other):
        other.strenght -= self.attackStr
        
class Infantry(Unit):

    def __repr__(self):
        return "Infantry "+self.name

class Archer(Unit):
    def __init__(self):
        self.coord = (0, 0)
        self.strenght = 5
        self.attackStr = 2
        self.name = ChrCounter().get_id()

    def __repr__(self):
        return "Archer "+self.name
        
class Chievalery(Unit):
    def __init__(self):
        self.coord = (0, 0)
        self.strenght = 15
        self.attackStr = 2
        self.name = ChrCounter().get_id()
        
    def move(self, direction):
        self.coord += 2 * direction

    def __repr__(self):
        return "Chivalery "+self.name
        
class Army:
    def __init__(self):
        self.army = []
        
    def addUnit(self, unit: Unit):
        self.army.append(unit)
        
# Строитель армии. Он умеет только формировать армию из создаваемых подразделений.
class ArmyBuilder:
    
    def __init__(self):
        self.p = None
        
    def createArmy(self):
        self.p = Army()
        
    def buildInfantry(self):
        self.p.addUnit(Infantry())
        
    def buildArcher(self):
        self.p.addUnit(Archer())
    
    def buildChievalery(self):
        self.p.addUnit(Chievalery())
    
    def getArmy(self):
        r = self.p
        self.p = None
        return r

# Этот класс умеет создавать подразделения и добавлять их в армию. 
# Возвращает готовую армию, построенную ArmyBuilder.
class Director:
    def createArmy(self, builder: ArmyBuilder) -> Army:
        builder.createArmy()
        builder.buildInfantry()
        builder.buildInfantry()
        builder.buildInfantry()
        builder.buildArcher()
        builder.buildArcher()
        builder.buildChievalery()
        return builder.getArmy()


In [439]:

a = Director().createArmy(ArmyBuilder())
a.army

[Infantry UD, Infantry UE, Infantry UF, Archer UG, Archer UH, Chivalery UI]